<a href="https://colab.research.google.com/github/kekecatf/Colab/blob/main/MakinaModelSon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1 Gerekli Kütüphanelerin Yüklenmesi
!pip install osmnx
!pip install networkx
!pip install joblib
!pip install folium requests

In [ ]:
#2 İçe Aktarılması
import joblib
import osmnx as ox
import pandas as pd
import networkx as nx
import zipfile
import folium

In [ ]:
#3 ZIP Dosyasını Açma
zip_path = '/content/drive/My Drive/random_forest_model.zip'
extract_to = '/content/drive/My Drive/veriler/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [ ]:
#4 Model Yükleme
model = joblib.load('/content/drive/My Drive/veriler/random_forest_model.pkl')
G = ox.load_graphml('/content/drive/My Drive/veriler/istanbul_yol_agi.graphml')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
#5 Hız Limitleri ve Özelliklerin Eklenmesi
speed_limits = {
    "residential": 50,
    "living_street": 50,
    "secondary": 80,
    "tertiary": 80,
    "unclassified": 0,
    "primary": 90,
    "trunk": 90,
    "motorway": 120
}

features = []

for u, v, data in G.edges(data=True):
    highway_type = data.get("highway", "residential")
    if isinstance(highway_type, list):
        highway_type = highway_type[0]
    data['speed_kph'] = speed_limits.get(highway_type, 50)
    features.append({
        'length': data.get('length', 0),
        'speed_kph': data.get('speed_kph', 50),
        'traffic_density': data.get('traffic_density', 0),
        'average_speed': data.get('average_speed', 50)
    })


In [ ]:
#6 Özelliklerin Veri Çerçevesine Dönüştürülmesi ve Tahmin
features_df = pd.DataFrame(features)
travel_times = model.predict(features_df)

for (u, v, data), travel_time in zip(G.edges(data=True), travel_times):
    data['travel_time'] = travel_time


In [ ]:
#7 Başlangıç ve Bitiş Noktalarının Belirlenmesi
latitude1, longitude1 = 41.211630, 28.914143  # Başlangıç koordinatları
latitude2, longitude2 = 41.015754, 29.134604  # Bitiş koordinatları

source = ox.distance.nearest_nodes(G, longitude1, latitude1)
target = ox.distance.nearest_nodes(G, longitude2, latitude2)


In [ ]:
#8 En Verimli Rota Ağırlıklandırması
for u, v, data in G.edges(data=True):
    length = data.get('length', 0)  # Yolun uzunluğu (metre)
    traffic = data.get('traffic_density', 0)  # Trafik yoğunluğu
    red_lights = data.get('red_light', 0)  # Kırmızı ışık sayısı

    # Yakıt tüketimini etkileyen faktörlerin ağırlıklı toplamı
    data['fuel_efficiency_cost'] = (
        length / 1000 +          # Mesafenin etkisi
        2 * traffic +            # Trafik yoğunluğu etkisi (katsayı ayarlanabilir)
        3 * red_lights           # Kırmızı ışıkların etkisi (katsayı ayarlanabilir)
    )


In [ ]:
#9 En Verimli Rota ve Toplam Maliyet
shortest_path = nx.shortest_path(G, source, target, weight='fuel_efficiency_cost')


In [ ]:
#10 Toplam Maliyet Hesaplama
total_cost = sum(
    G[u][v][0].get('fuel_efficiency_cost', 0) for u, v in zip(shortest_path[:-1], shortest_path[1:])
)

# Toplam Mesafe ve Tahmini Süre
total_distance_km = sum(
    G[u][v][0].get('length', 0) for u, v in zip(shortest_path[:-1], shortest_path[1:])
) / 1000  # Metreden kilometreye dönüştürme

average_speed = 50  # Varsayılan ortalama hız (km/saat)
estimated_time_minutes = (total_distance_km / average_speed) * 60  # Dakika cinsinden süre

# Bilgilerin Konsola Yazdırılması
print(f"Yakıt verimliliği rotasının toplam maliyeti: {total_cost:.2f}")
print(f"Toplam mesafe: {total_distance_km:.2f} km")
print(f"Tahmini süre: {estimated_time_minutes:.2f} dakika")


Yakıt verimliliği rotasının toplam maliyeti: 36.74
Toplam mesafe: 36.74 km
Tahmini süre: 44.09 dakika


In [ ]:
#11 Harita Görselleştirme
m = folium.Map(location=[latitude1, longitude1], zoom_start=12)


In [ ]:
#12 Başlangıç ve bitiş noktalarını işaretleme
folium.Marker(
    location=[latitude1, longitude1],
    tooltip="Başlangıç",
    icon=folium.Icon(color="green")
).add_to(m)

folium.Marker(
    location=[latitude2, longitude2],
    tooltip="Bitiş",
    icon=folium.Icon(color="red")
).add_to(m)


In [ ]:
#13 Rota Çizimi
path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
folium.PolyLine(
    locations=path_coords,
    color="blue",
    weight=5,
    opacity=0.8
).add_to(m)

In [ ]:
#14 Rota Bilgileri Popup (Bilgi Balonu) Eklenmesi
popup_text = (
    f"<b>Rota Bilgileri:</b><br>"
    f"Toplam mesafe: {total_distance_km:.2f} km<br>"
    f"Tahmini süre: {estimated_time_minutes:.2f} dakika<br>"
    f"Yakıt verimliliği maliyeti: {total_cost:.2f}"
)
folium.Marker(
    location=[(latitude1 + latitude2) / 2, (longitude1 + longitude2) / 2],  # Rota ortasına yerleştirme
    popup=folium.Popup(popup_text, max_width=300),
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(m)


In [ ]:
#15 Haritayı Kaydetme ve Görüntüleme
m.save("route_map.html")
m